In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stats
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

In [4]:
data=pd.read_csv("train.csv")

In [5]:
pd.crosstab(data.Age, data.Survived)

Survived,0,1
Age,,
0.42,0,1
0.67,0,1
0.75,0,2
0.83,0,2
0.92,0,1
...,...,...
70.00,2,0
70.50,1,0
71.00,2,0


In [6]:
print(data.isnull().sum())
data['Age'].fillna(data['Age'].mean(),inplace=True)
data['Embarked'].fillna(data['Embarked'].mode()[0],inplace=True)
data['Fare'].fillna(data['Fare'].mean(),inplace=True)
data['Cabin']=data['Cabin'].fillna("Unknown")
data['Deck']=data['Cabin'].str.get(0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [7]:
data['Title'] = data.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

In [8]:
from sklearn.preprocessing import LabelEncoder 
le= LabelEncoder()

data['Sex']=le.fit_transform(data['Sex'])
data['Embarked']=le.fit_transform(data['Embarked'])

data['Title'] = data['Title'].replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer')
data['Title'] = data['Title'].replace(['Don', 'Sir', 'the Countess', 'Dona', 'Lady'], 'Royalty')
data['Title'] = data['Title'].replace('Jonkheer', 'Master')
data['Title'] = data['Title'].replace('Mlle', 'Miss')
data['Title'] = data['Title'].replace('Ms', 'Miss')
data['Title'] = data['Title'].replace('Mme', 'Mrs')
data['Title']=le.fit_transform(data['Title'])
   
data['Fare']=(data['Fare']-data['Fare'].min())/(data['Fare'].max()-data['Fare'].min())

data['FamilySize']=data['Parch']+data['SibSp']+1

data['Deck']=le.fit_transform(data['Deck'])

data['TravelAlone']=np.where(data['FamilySize']>1, 0, 1)

Ticket_Count = dict(data['Ticket'].value_counts())

data['TicketGroup'] = data['Ticket'].apply(lambda x:Ticket_Count[x])    

In [9]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Deck,Title,FamilySize,TravelAlone,TicketGroup
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,0.014151,Unknown,2,8,3,2,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,0.139136,C85,0,2,4,2,0,1
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,0.015469,Unknown,2,8,2,1,1,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,0.103644,C123,2,2,4,2,0,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,0.015713,Unknown,2,8,3,1,1,1


In [10]:
data=data.drop(['PassengerId','Ticket',"Cabin",'Name','SibSp','Parch'],axis='columns')

In [11]:
X=data.drop(['Survived'],axis='columns').values
Y=data['Survived'].values
X_train,X_test,y_train,y_test=train_test_split(X,Y,test_size=0.2, random_state=123)

In [12]:
model_LR=LogisticRegression()
model_LR.fit(X_train,y_train)

C:\Users\hp\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [13]:
pred_LR=model_LR.predict(X_test)
print(confusion_matrix(y_test,pred_LR))
accuracy_score(y_test,pred_LR)

[[96 18]
 [14 51]]


0.8212290502793296

In [14]:
model_RFC_base=RandomForestClassifier()
model_RFC_base.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [15]:
pred_RFC_base=model_RFC_base.predict(X_test)
print(confusion_matrix(y_test,pred_RFC_base))
accuracy_score(y_test,pred_RFC_base)

[[98 16]
 [12 53]]


0.8435754189944135

In [16]:
random_grid = {
                 'bootstrap': [True, False],
 'max_depth': [5, 10, 20, 30, 40, 50],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [50,100,200,400]
                }

In [17]:
clf=RandomForestClassifier()
model_RFC_rand=RandomizedSearchCV(estimator = clf, param_distributions = random_grid, cv = 3, verbose=2, n_jobs = -1)
model_RFC_rand.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.6s finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [18]:
model_RFC_rand.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 50,
 'bootstrap': True}

In [19]:
model_best_rand= model_RFC_rand.best_estimator_
pred_best_rand=model_best_rand.predict(X_test)
print(confusion_matrix(y_test,pred_best_rand))
accuracy_score(y_test,pred_best_rand)

[[102  12]
 [ 14  51]]


0.8547486033519553

In [20]:
grid_param= {
    'bootstrap': [True],
    'max_depth': [3,4,5,6,7,8],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1,2,3],
    'min_samples_split': [8,9,10,11,12],
    'n_estimators': [60,70,80,90,100,110,120,130,140]
}

In [21]:
clf1=RandomForestClassifier()
model_grid = GridSearchCV(estimator = clf1, param_grid = grid_param, cv = 3, n_jobs = -1, verbose = 2)
model_grid.fit(X_train,y_train)

Fitting 3 folds for each of 810 candidates, totalling 2430 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   16.6s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:   59.9s
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 2430 out of 2430 | elapsed:  3.5min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [22]:
model_grid.best_params_

{'bootstrap': True,
 'max_depth': 6,
 'max_features': 'sqrt',
 'min_samples_leaf': 3,
 'min_samples_split': 12,
 'n_estimators': 80}

In [23]:
model_best_grid = model_grid.best_estimator_
pred_grid=model_best_grid.predict(X_test)
print(confusion_matrix(y_test,pred_grid))
accuracy_score(y_test,pred_grid)

[[103  11]
 [ 15  50]]


0.8547486033519553

In [24]:
model_RFC_boot=RandomForestClassifier(bootstrap= True,max_depth= 5, max_features= 'sqrt', min_samples_leaf= 3,min_samples_split= 8, n_estimators= 100)
model_RFC_boot.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=3, min_samples_split=8,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [25]:
pred_boot=model_RFC_boot.predict(X_test)
print(confusion_matrix(y_test,pred_boot))
accuracy_score(y_test,pred_boot)

[[104  10]
 [ 17  48]]


0.8491620111731844